In [ ]:
# Cell 1: Imports & Constants
import os
import numpy as np
import tensorflow as tf
import wandb
from wandb.integration.keras import WandbCallback
from tensorflow.keras import layers, models, callbacks, regularizers
from keras.saving import register_keras_serializable

# Constants
NUM_CROPS = 9
NUM_DISEASES = 33
BATCH_SIZE = 128
INPUT_SHAPE = (224, 224, 3)

2025-03-25 12:03:08.808953: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-25 12:03:08.853856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742884388.913251    8979 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742884388.929395    8979 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-25 12:03:08.984638: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
# Cell 2: W&B Init
wandb.init(
    project="plantvillage_simclr_dann",
    name="simclr_dann_fusion_run_6",
    config={
        "architecture": "SimCLR + DANN",
        "encoder": "SimCLR-EfficientNetV2B0",
        "optimizer": "Adam",
        "learning_rate": 1e-4,
        "batch_size": BATCH_SIZE,
        "loss_weights": {"disease_output": 2.0, "domain_output": 1.0},
        "freeze_encoder": True,
        "input_shape": INPUT_SHAPE
    }
)
CONFIG = wandb.config

wandb: Currently logged in as: gsrivatsa1398 (gsrivatsa1398-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
# Cell 3: Dataset Paths
DATASET_DIR = "/home/srivatsa/Multiple_Crop_Disease_Detection/Dataset/PlantVillage_Structured/AUG_TFRecord/split"
train_tfrecord = os.path.join(DATASET_DIR, "train.tfrecord")
val_tfrecord = os.path.join(DATASET_DIR, "val.tfrecord")
test_tfrecord = os.path.join(DATASET_DIR, "test.tfrecord")

In [ ]:
# Cell 4: Dataset Load Functions
feature_description = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "crop": tf.io.FixedLenFeature([], tf.int64),
    "disease": tf.io.FixedLenFeature([], tf.int64)
}

def parse_example(example_proto):
    example = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.image.decode_jpeg(example['image'], channels=3)
    image = tf.image.resize(image, [224, 224]) / 255.0
    return image, {
        "disease_output": tf.one_hot(example['disease'], NUM_DISEASES),
        "domain_output": tf.one_hot(example['crop'], NUM_CROPS)
    }

def load_dataset(path):
    dataset = tf.data.TFRecordDataset(path)
    dataset = dataset.map(parse_example, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = load_dataset(train_tfrecord)
val_dataset = load_dataset(val_tfrecord)
test_dataset = load_dataset(test_tfrecord)


I0000 00:00:1742884398.152956    8979 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3573 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [ ]:
# Cell 5: Load SimCLR Encoder
encoder_path = "/home/srivatsa/Multiple_Crop_Disease_Detection/Models/SimCLR_Pretrained/best_simCLR_Encoder.keras"
simclr_encoder = tf.keras.models.load_model(encoder_path, compile=False)
if CONFIG.freeze_encoder:
    simclr_encoder.trainable = False

In [ ]:
@register_keras_serializable()
class GradientReversal(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(GradientReversal, self).__init__(**kwargs)

    def call(self, x):
        @tf.custom_gradient
        def reverse_gradient(x):
            def grad(dy):
                return -dy
            return x, grad
        return reverse_gradient(x)

    def get_config(self):
        config = super().get_config()
        return config

In [ ]:
# Cell 7: Build SimCLR + DANN Model
inputs = layers.Input(shape=INPUT_SHAPE)
features = simclr_encoder(inputs)

# Disease Classifier
x = layers.Dense(256, activation='relu')(features)
x = layers.Dropout(0.3)(x)
disease_output = layers.Dense(NUM_DISEASES, activation='softmax', name='disease_output')(x)

# Domain Discriminator
grl = GradientReversal()(features)
d = layers.Dense(128, activation='relu')(grl)
d = layers.Dropout(0.3)(d)
domain_output = layers.Dense(NUM_CROPS, activation='softmax', name='domain_output')(d)

model = models.Model(inputs=inputs, outputs=[disease_output, domain_output])

In [ ]:
# Cell 8: Compile Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(CONFIG.learning_rate),
    loss={
        "disease_output": "categorical_crossentropy",
        "domain_output": "categorical_crossentropy"
    },
    loss_weights={
        "disease_output": CONFIG.loss_weights["disease_output"],
        "domain_output": CONFIG.loss_weights["domain_output"]
    },
    metrics={
        "disease_output": "accuracy",
        "domain_output": "accuracy"
    }
)

In [ ]:
# Cell 9: Callbacks
cb = [
    callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor="val_disease_output_loss", factor=0.5, patience=3, verbose=1),
    callbacks.ModelCheckpoint("/home/srivatsa/Multiple_Crop_Disease_Detection/Models/simCLR_DANN_Fusion/best_fusion_model/best_simCLR_DANN_fusionModel.keras", monitor="val_disease_output_accuracy", save_best_only=True),
    WandbCallback(log_model=False, save_model=False, save_graph=False)
]


wandb: WARNING WandbCallback is deprecated and will be removed in a future release. Please use the WandbMetricsLogger, WandbModelCheckpoint, and WandbEvalCallback callbacks instead. See https://docs.wandb.ai/guides/integrations/keras for more information.


In [ ]:
# Cell 10: Train Model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=300,
    callbacks=cb
)


Epoch 1/300


2025-03-25 12:03:33.290978: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:370] TFRecordDataset `buffer_size` is unspecified, default to 262144
I0000 00:00:1742884413.736384    9141 service.cc:148] XLA service 0x7ff01805cb50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742884413.737440    9141 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-03-25 12:03:34.125743: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742884415.810158    9141 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-25 12:03:38.577144: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12548', 132 bytes spill stores, 132 bytes spill loads

2025-03-25 12:03:38.970016: I ex

    248/Unknown 55s 94ms/step - disease_output_accuracy: 0.3095 - disease_output_loss: 2.7091 - domain_output_accuracy: 0.4785 - domain_output_loss: 1.5627 - loss: 6.9810 

2025-03-25 12:04:23.857555: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 84 bytes spill stores, 84 bytes spill loads

2025-03-25 12:04:24.034269: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1', 80 bytes spill stores, 80 bytes spill loads

2025-03-25 12:04:24.284277: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1', 88 bytes spill stores, 88 bytes spill loads

2025-03-25 12:04:24.295853: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_12548', 760 bytes spill stores, 760 bytes spill loads

2025-03-25 12:04:24.594007: I external/local_xla/xla/stream_executor

    249/Unknown 72s 160ms/step - disease_output_accuracy: 0.3101 - disease_output_loss: 2.7069 - domain_output_accuracy: 0.4790 - domain_output_loss: 1.5613 - loss: 6.9751

...


Epoch 207: ReduceLROnPlateau reducing learning rate to 3.8146971692576415e-10.
249/249 ━━━━━━━━━━━━━━━━━━━━ 31s 123ms/step - disease_output_accuracy: 0.9661 - disease_output_loss: 0.1119 - domain_output_accuracy: 0.9770 - domain_output_loss: 0.0716 - loss: 0.2955 - val_disease_output_accuracy: 0.9438 - val_disease_output_loss: 0.1795 - val_domain_output_accuracy: 0.9686 - val_domain_output_loss: 0.1047 - val_loss: 0.4636 - learning_rate: 7.6294e-10
Epoch 208/300


2025-03-25 13:51:06.928151: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 12690982651069564249
2025-03-25 13:51:06.928224: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 11035634526328842720
2025-03-25 13:51:07.090567: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 77091840 bytes after encountering the first element of size 77091840 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


248/249 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - disease_output_accuracy: 0.9629 - disease_output_loss: 0.1181 - domain_output_accuracy: 0.9779 - domain_output_loss: 0.0686 - loss: 0.3048 

2025-03-25 13:51:31.432884: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 77091840 bytes after encountering the first element of size 77091840 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


249/249 ━━━━━━━━━━━━━━━━━━━━ 31s 123ms/step - disease_output_accuracy: 0.9629 - disease_output_loss: 0.1181 - domain_output_accuracy: 0.9780 - domain_output_loss: 0.0686 - loss: 0.3047 - val_disease_output_accuracy: 0.9438 - val_disease_output_loss: 0.1795 - val_domain_output_accuracy: 0.9686 - val_domain_output_loss: 0.1046 - val_loss: 0.4636 - learning_rate: 3.8147e-10


In [ ]:
# Cell 11: Evaluate Final Model
results = model.evaluate(test_dataset, return_dict=True)
print("\n✅ Final Evaluation Metrics:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")

print("\n✅ SimCLR + DANN Phase 4 Model Training Completed!")


      2/Unknown 0s 78ms/step - disease_output_accuracy: 0.9121 - disease_output_loss: 0.2382 - domain_output_accuracy: 0.9512 - domain_output_loss: 0.1937 - loss: 0.6702 

2025-03-25 13:51:38.173184: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 77091840 bytes after encountering the first element of size 77091840 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


     35/Unknown 4s 112ms/step - disease_output_accuracy: 0.9378 - disease_output_loss: 0.1941 - domain_output_accuracy: 0.9625 - domain_output_loss: 0.1302 - loss: 0.5183

2025-03-25 13:51:44.311652: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1', 80 bytes spill stores, 80 bytes spill loads

2025-03-25 13:51:44.343914: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 84 bytes spill stores, 84 bytes spill loads



36/36 ━━━━━━━━━━━━━━━━━━━━ 16s 436ms/step - disease_output_accuracy: 0.9379 - disease_output_loss: 0.1943 - domain_output_accuracy: 0.9626 - domain_output_loss: 0.1291 - loss: 0.5177

✅ Final Evaluation Metrics:
disease_output_accuracy: 0.9390
disease_output_loss: 0.1993
domain_output_accuracy: 0.9641
domain_output_loss: 0.1110
loss: 0.5083

✅ SimCLR + DANN Phase 4 Model Training Completed!
